In [35]:
# Call libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
from shapely import geometry
import libpysal
import time
from rasterstats import zonal_stats

import warnings
warnings.filterwarnings("ignore")

In [36]:
# WGS84和GJC坐标系转换函数
# -*- coding: utf-8 -*-
import json
import urllib
import math
# import numpy as np

x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626  # π
a = 6378245.0  # 长半轴
ee = 0.00669342162296594323  # 偏心率平方


'''
输入（经度，维度）
'''
def bd09_to_gcj02(bd_lon, bd_lat):
    """
    百度坐标系(BD-09)转火星坐标系(GCJ-02)
    百度——>谷歌、高德
    :param bd_lat:百度坐标纬度
    :param bd_lon:百度坐标经度
    :return:转换后的坐标列表形式
    """
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]
def gcj02_to_wgs84(lng, lat):
    """
    GCJ02(火星坐标系)转GPS84
    :param lng:火星坐标系的经度
    :param lat:火星坐标系纬度
    :return:
    """
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [lng * 2 - mglng, lat * 2 - mglat]
def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)
def bd09_to_wgs84(bd_lon, bd_lat):
    lon, lat = bd09_to_gcj02(bd_lon, bd_lat)
    return gcj02_to_wgs84(lon, lat)
def gcj02_to_bd09(lng, lat):
    """
    火星坐标系(GCJ-02)转百度坐标系(BD-09)
    谷歌、高德——>百度
    :param lng:火星坐标经度
    :param lat:火星坐标纬度
    :return:
    """
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]
def wgs84_to_gcj02(lng, lat):
    """
    WGS84转GCJ02(火星坐标系)
    :param lng:WGS84坐标系的经度
    :param lat:WGS84坐标系的纬度
    :return:
    """
    if out_of_china(lng, lat):  # 判断是否在国内
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]
def wgs84_to_bd09(lon, lat):
    lon, lat = wgs84_to_gcj02(lon, lat)
    return gcj02_to_bd09(lon, lat)

def out_of_china(lng, lat):
    """
    判断是否在国内，不在国内不做偏移
    :param lng:
    :param lat:
    :return:
    """
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret
def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret


In [32]:
# 转换WGS84坐标为GCJ02的函数
def transform_coordinates(row):
    lng_x, lat_x = wgs84_to_gcj02(row['longitude_x'], row['latitude_x'])
    lng_y, lat_y = wgs84_to_gcj02(row['longitude_y'], row['latitude_y'])
    return pd.Series({'lng_x_gcj02': lng_x, 'lat_x_gcj02': lat_x, 'lng_y_gcj02': lng_y, 'lat_y_gcj02': lat_y})

In [14]:
# 按照经纬度计算距离的函数
from math import radians, cos, sin, asin, sqrt
 
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000

In [6]:
pop_resid = gpd.read_file('residential_pop.shp') 
pop_resid = pop_resid.to_crs(epsg = 4326)

pop_resid = pop_resid[(pop_resid['sum_pop'] >= 1)]
pop_resid['cen'] = pop_resid['geometry'].centroid
pop_resid['Popid'] = pop_resid.index + 1

pop_resid['longitude'] = pop_resid['cen'].x
pop_resid['latitude'] = pop_resid['cen'].y

pop_resid

In [10]:
community_park = gpd.read_file('community_park_entrance.geojson').to_crs(epsg = 4326)
city_park = gpd.read_file('city_park_entrance.geojson').to_crs(epsg = 4326)
nature_park = gpd.read_file('nature_park_entrance.geojson').to_crs(epsg = 4326)

## Walking

1.公园点-popcell质心，直线距离<=10min，高德步行速度75m/min，即750m,的OD对；  
2.高德API计算distance，cost；  
3.Ga2SFCA计算可达性。

In [1]:
# 调用高德路径规划api-步行，计算路径距离distance（米），耗时cost（秒）
# 路径规划1.0
import json
from urllib import request

def fetch_GaodeMap_walking(df):
    urlbase = 'https://restapi.amap.com/v3/direction/walking?&key=225cfe7d506a6037debd6a9f4d5aa583&origin={0},{1}&destination={2},{3}'
    distances = []
    costs = []
    exceptions = []  # 用来收集引发异常的行索引

    for i, row in df.iterrows():
        x1 = row['lng_x_gcj02']
        y1 = row['lat_x_gcj02']
        x2 = row['lng_y_gcj02']
        y2 = row['lat_y_gcj02']
        url = urlbase.format(x1, y1, x2, y2)

        try:
            html = request.urlopen(url, timeout=15).read()
            js = json.loads(html)
            distance = js['route']['paths'][0]['distance']
            cost = js['route']['paths'][0]['duration']
        except Exception as e:
            print(f"Error processing row {i}: {e}")
            distance = None
            cost = None
            exceptions.append(i)  # 将引发异常的行索引添加到列表中

        distances.append(distance)
        costs.append(cost)

    df['distance'] = distances
    df['cost'] = costs

    return df, exceptions

### a) Walking——Community park——Travel time thresholds =10min   

In [19]:
# 1 连接pop和park
pop_resid['index'] = 1
community_park['index'] = 1
OD_a = pd.merge(pop_resid, community_park, on='index').drop('index', axis=1)

In [20]:
len(OD_a)

575820

In [21]:
# 2 根据OD左边计算直线距离，得到新列length
OD_a['length'] = OD_a[['longitude_x', 'latitude_x','longitude_y', 'latitude_y',]].\
apply(lambda x:haversine(x[0],x[1],x[2],x[3]),axis=1)

In [26]:
# 3 筛选lenth<=750m的OD对
f_OD_a = OD_a[(OD_a['length'] <= 750)]

# 只保留同一个公园直线距离最近的入口
idx = f_OD_a.groupby(['Popid', 'Coparkid'])['length'].idxmin()
f_OD_a = f_OD_a.loc[idx]

In [27]:
len(f_OD_a)

2135

In [33]:
# 4 转换WGS84坐标为GCJ02，便于下一步高德api计算
new_columns = f_OD_a.apply(transform_coordinates, axis=1)
# 合并原始 DataFrame 和新列
f_OD_a_GCJ02 = pd.concat([f_OD_a, new_columns], axis=1)

In [34]:
len(f_OD_a_GCJ02)

2135

In [37]:
# 5 使用api计算高德地图时间距离
start_time = time.time()

ODa_Gaode, exception_rows = fetch_GaodeMap_walking(f_OD_a_GCJ02)
print("Exception rows:", exception_rows)

end_time = time.time()
execution_time = (end_time - start_time)/60
print("Run time: ", execution_time, "mins")

Exception rows: []
Run time:  28.994334121545155 mins


In [ ]:
# # 6 检查问题行
# exception_df = f_OD_a_GCJ02.loc[exception_rows]

# # Calculate Gaode map for exception_df travel time and travel distance again
# exception_df_re,exception = fetch_GaodeMap_walking(exception_df)

# ODa_Gaode['distance'] = ODa_Gaode['distance'].combine_first(exception_df_re['distance'])
# ODa_Gaode['cost'] = ODa_Gaode['cost'].combine_first(exception_df_re['cost'])

In [38]:
print(ODa_Gaode.isnull().sum())

name           416
sum_pop          0
geometry_x       0
cen              0
Popid            0
longitude_x      0
latitude_x       0
area             0
Coparkid         0
polygonStr       0
avg_area         0
longitude_y      0
latitude_y       0
geometry_y       0
length           0
lng_x_gcj02      0
lat_x_gcj02      0
lng_y_gcj02      0
lat_y_gcj02      0
distance         0
cost             0
dtype: int64


In [39]:
# 7 转换为数
ODa_Gaode['cost'] = ODa_Gaode['cost'].astype(float)
ODa_Gaode['distance'] = ODa_Gaode['distance'].astype(float)

In [40]:
ODa_Gaode.to_csv('tmp/ODa_Gaode.csv',index=False,encoding='utf-8')

In [41]:
# 8 以10min，即600s为阈值，计算Ga2SFCA,得到每个popcell的可达性
f_ODa_Gaode = ODa_Gaode[ODa_Gaode['cost'] <= 600]

f_ODa_Gaode.reset_index(drop = True, inplace = True)

len(f_ODa_Gaode)
len(f_ODa_Gaode['Popid'].unique())

608

In [42]:
# 9 定义函数
def Ga_600s(dij):
    e=math.exp(1)
    g=(e**(-0.5*(dij/600)**2)-e**(-0.5))/(1-e**(-0.5)) # dij是供给和需求之间的长度（时间cost or 距离distance），600s是阈值（时长or距离）
    return g

def Get_Rj(x):
    x=x.reset_index()
    Sj=x['area'][0]
    
    dt=0
    for i in range(len(x)):
        vl=x['sum_pop'][i]*Ga_600s(x['cost'][i])
        dt=dt+vl
    return Sj/dt

def Get_Ai(x):
    x=x.reset_index()
    
    dt=0
    for i in range(len(x)):
        vl=x['Rj'][i]*Ga_600s(x['cost'][i])
        dt=dt+vl
    return dt

In [43]:
# 10 step1 计算每个公园的Rj
# 公园面积 ➗ 该公园搜索阈以内的人口数

park_s1 = f_ODa_Gaode.groupby(by='Coparkid').apply(Get_Rj).reset_index()
park_s1 = park_s1.rename(columns={0: 'Rj'})

f_ODa_Gaode = pd.merge(f_ODa_Gaode,park_s1 [['Coparkid', 'Rj']], on='Coparkid', how='left')

In [44]:
# 11 step2 计算每个pop cell的Ai
# 每个供给点popcell,搜索阈以内的Rj,即park_v1
pop_s1 = f_ODa_Gaode.groupby(by='Popid').apply(Get_Ai).reset_index()
pop_s1 = pop_s1.rename(columns={0: 'Ai'})

f_ODa_Gaode = pd.merge(f_ODa_Gaode,pop_s1[['Popid', 'Ai']], on='Popid', how='left')

In [45]:
# 12 简化
access_socre_a = f_ODa_Gaode[['Popid', 'sum_pop', 'geometry_x', 'Ai']]
access_socre_a = access_socre_a.rename(columns={'geometry_x':'geometry'})
access_socre_a.drop_duplicates(inplace=True)
len(access_socre_a)

608

In [48]:
result_a = pop_resid.merge(access_socre_a[['Popid', 'Ai']], on='Popid', how='left')

In [53]:
result_a.head()

,name,sum_pop,geometry,cen,Popid,longitude,latitude,index,Ai
0,None,7276.136558,"POLYGON ((114.46834 22.60207, 114.46765 22.601...",POINT (114.47232 22.59552),1,114.472319,22.595523,1,2.924363
1,None,1217.979429,"POLYGON ((114.49399 22.53761, 114.49294 22.535...",POINT (114.48832 22.53681),2,114.488317,22.536814,1,NaN
2,?????,2048.236732,"POLYGON ((114.17310 22.64141, 114.17391 22.641...",POINT (114.17423 22.63940),4,114.174231,22.639398,1,NaN
3,None,11243.644833,"POLYGON ((114.10559 22.61673, 114.10483 22.616...",POINT (114.10326 22.61540),5,114.103263,22.615398,1,NaN
4,None,875.631427,"POLYGON ((114.18524 22.59461, 114.18549 22.595...",POINT (114.18960 22.59443),6,114.189601,22.594434,1,NaN


In [54]:
result_a = result_a[['sum_pop', 'geometry', 'Popid', 'Ai']]

In [55]:
result_a.to_file('tmp/result_a.geojson')

### b) Walking——City park——Travel time thresholds =20min 

In [56]:
# 1 连接pop和park
pop_resid['index'] = 1
city_park['index'] = 1
OD_b = pd.merge(pop_resid, city_park, on='index').drop('index', axis=1)

In [57]:
len(OD_b)

754964

In [58]:
# 2 根据OD左边计算直线距离，得到新列length
OD_b['length'] = OD_b[['longitude_x', 'latitude_x','longitude_y', 'latitude_y',]].\
apply(lambda x:haversine(x[0],x[1],x[2],x[3]),axis=1)

In [59]:
# 3 筛选lenth<=1500m的OD对
f_OD_b = OD_b[(OD_b['length'] <= 1500)]

# 只保留同一个公园直线距离最近的入口
idx = f_OD_b.groupby(['Popid', 'Cityparkid'])['length'].idxmin()
f_OD_b = f_OD_b.loc[idx]

In [60]:
len(f_OD_b)

2150

In [61]:
# 4 转换WGS84坐标为GCJ02，便于下一步高德api计算
new_columns = f_OD_b.apply(transform_coordinates, axis=1)
# 合并原始 DataFrame 和新列
f_OD_b_GCJ02 = pd.concat([f_OD_b, new_columns], axis=1)

In [62]:
len(f_OD_b_GCJ02)

2150

In [63]:
# 5 使用api计算高德地图时间距离
start_time = time.time()

ODb_Gaode, exception_rows = fetch_GaodeMap_walking(f_OD_b_GCJ02)
print("Exception rows:", exception_rows)

end_time = time.time()
execution_time = (end_time - start_time)/60
print("Run time: ", execution_time, "mins")

Error processing row 317716: <urlopen error EOF occurred in violation of protocol (_ssl.c:997)>
Error processing row 350004: <urlopen error EOF occurred in violation of protocol (_ssl.c:997)>
Error processing row 389174: <urlopen error EOF occurred in violation of protocol (_ssl.c:997)>
Exception rows: [317716, 350004, 389174]
Run time:  28.838023257255553 mins


In [65]:
# 6 检查问题行
exception_df = f_OD_b_GCJ02.loc[exception_rows]

# Calculate Gaode map for exception_df travel time and travel distance again
exception_df_re,exception = fetch_GaodeMap_walking(exception_df)

ODb_Gaode['distance'] = ODb_Gaode['distance'].combine_first(exception_df_re['distance'])
ODb_Gaode['cost'] = ODb_Gaode['cost'].combine_first(exception_df_re['cost'])

In [66]:
print(ODb_Gaode.isnull().sum())

name           461
sum_pop          0
geometry_x       0
cen              0
Popid            0
longitude_x      0
latitude_x       0
area             0
Cityparkid       0
count            0
avg_area         0
longitude_y      0
latitude_y       0
polygonStr       0
geometry_y       0
length           0
lng_x_gcj02      0
lat_x_gcj02      0
lng_y_gcj02      0
lat_y_gcj02      0
distance         0
cost             0
dtype: int64


In [72]:
# 7 转换为数
ODb_Gaode['cost'] = ODb_Gaode['cost'].astype(float)
ODb_Gaode['distance'] = ODb_Gaode['distance'].astype(float)

In [73]:
ODb_Gaode.to_csv('tmp/ODb_Gaode.csv',index=False,encoding='utf-8')

In [74]:
# 8 以20min，即1200s为阈值，计算Ga2SFCA,得到每个popcell的可达性
f_ODb_Gaode = ODb_Gaode[ODb_Gaode['cost'] <= 1200]

f_ODb_Gaode.reset_index(drop = True, inplace = True)

len(f_ODb_Gaode)
len(f_ODb_Gaode['Popid'].unique())

907

In [75]:
# 9 定义函数
def Ga_1200s(dij):
    e=math.exp(1)
    g=(e**(-0.5*(dij/1200)**2)-e**(-0.5))/(1-e**(-0.5)) # dij是供给和需求之间的长度（时间cost or 距离distance），600s是阈值（时长or距离）
    return g

def Get_Rj(x):
    x=x.reset_index()
    Sj=x['area'][0]
    
    dt=0
    for i in range(len(x)):
        vl=x['sum_pop'][i]*Ga_1200s(x['cost'][i])
        dt=dt+vl
    return Sj/dt

def Get_Ai(x):
    x=x.reset_index()
    
    dt=0
    for i in range(len(x)):
        vl=x['Rj'][i]*Ga_1200s(x['cost'][i])
        dt=dt+vl
    return dt

In [76]:
# 10 step1 计算每个公园的Rj
# 公园面积 ➗ 该公园搜索阈以内的人口数

park_s2 = f_ODb_Gaode.groupby(by='Cityparkid').apply(Get_Rj).reset_index()
park_s2 = park_s2.rename(columns={0: 'Rj'})

f_ODb_Gaode = pd.merge(f_ODb_Gaode,park_s2 [['Cityparkid', 'Rj']], on='Cityparkid', how='left')

In [77]:
# 11 step2 计算每个pop cell的Ai
# 每个供给点popcell,搜索阈以内的Rj,即park_v1
pop_s2 = f_ODb_Gaode.groupby(by='Popid').apply(Get_Ai).reset_index()
pop_s2 = pop_s2.rename(columns={0: 'Ai'})

f_ODb_Gaode = pd.merge(f_ODb_Gaode,pop_s2[['Popid', 'Ai']], on='Popid', how='left')

In [78]:
# 12 简化
access_socre_b = f_ODb_Gaode[['Popid', 'sum_pop', 'geometry_x', 'Ai']]
access_socre_b = access_socre_b.rename(columns={'geometry_x':'geometry'})
access_socre_b.drop_duplicates(inplace=True)
len(access_socre_b)

907

In [79]:
result_b = pop_resid.merge(access_socre_b[['Popid', 'Ai']], on='Popid', how='left')

In [80]:
result_b = result_b[['sum_pop', 'geometry', 'Popid', 'Ai']]

In [81]:
result_b.to_file('tmp/result_b.geojson')

### c) Walking——Natural park——Travel time thresholds =30min

In [82]:
# 1 连接pop和park
pop_resid['index'] = 1
nature_park['index'] = 1
OD_c = pd.merge(pop_resid, nature_park, on='index').drop('index', axis=1)

In [83]:
len(OD_c)

162692

In [84]:
# 2 根据OD左边计算直线距离，得到新列length
OD_c['length'] = OD_c[['longitude_x', 'latitude_x','longitude_y', 'latitude_y',]].\
apply(lambda x:haversine(x[0],x[1],x[2],x[3]),axis=1)

In [85]:
# 3 筛选lenth<=2250m的OD对
f_OD_c = OD_c[(OD_c['length'] <= 2250)]

# 只保留同一个公园直线距离最近的入口
idx = f_OD_c.groupby(['Popid', 'Natureparkid'])['length'].idxmin()
f_OD_c = f_OD_c.loc[idx]

In [86]:
len(f_OD_c)

482

In [87]:
# 4 转换WGS84坐标为GCJ02，便于下一步高德api计算
new_columns = f_OD_c.apply(transform_coordinates, axis=1)
# 合并原始 DataFrame 和新列
f_OD_c_GCJ02 = pd.concat([f_OD_c, new_columns], axis=1)

In [88]:
len(f_OD_c_GCJ02)

482

In [89]:
# 5 使用api计算高德地图时间距离
start_time = time.time()

ODc_Gaode, exception_rows = fetch_GaodeMap_walking(f_OD_c_GCJ02)
print("Exception rows:", exception_rows)

end_time = time.time()
execution_time = (end_time - start_time)/60
print("Run time: ", execution_time, "mins")

Exception rows: []
Run time:  6.431661494572958 mins


In [ ]:
# # 6 检查问题行
# exception_df = f_OD_b_GCJ02.loc[exception_rows]

# # Calculate Gaode map for exception_df travel time and travel distance again
# exception_df_re,exception = fetch_GaodeMap_walking(exception_df)

# ODb_Gaode['distance'] = ODb_Gaode['distance'].combine_first(exception_df_re['distance'])
# ODb_Gaode['cost'] = ODb_Gaode['cost'].combine_first(exception_df_re['cost'])

In [90]:
print(ODc_Gaode.isnull().sum())

name            98
sum_pop          0
geometry_x       0
cen              0
Popid            0
longitude_x      0
latitude_x       0
area             0
Natureparkid     0
count            0
avg_area         0
longitude_y      0
latitude_y       0
polygonStr       0
geometry_y       0
length           0
lng_x_gcj02      0
lat_x_gcj02      0
lng_y_gcj02      0
lat_y_gcj02      0
distance         0
cost             0
dtype: int64


In [91]:
# 7 转换为数
ODc_Gaode['cost'] = ODc_Gaode['cost'].astype(float)
ODc_Gaode['distance'] = ODc_Gaode['distance'].astype(float)

In [92]:
ODc_Gaode.to_csv('tmp/ODc_Gaode.csv',index=False,encoding='utf-8')

In [115]:
# 8 以30min，即1800s为阈值，计算Ga2SFCA,得到每个popcell的可达性
f_ODc_Gaode = ODc_Gaode[ODc_Gaode['cost'] <= 1800]

f_ODc_Gaode.reset_index(drop = True, inplace = True)

len(f_ODc_Gaode)
len(f_ODc_Gaode['Popid'].unique())

288

In [116]:
# 9 定义函数
def Ga_1800s(dij):
    e=math.exp(1)
    g=(e**(-0.5*(dij/1800)**2)-e**(-0.5))/(1-e**(-0.5)) # dij是供给和需求之间的长度（时间cost or 距离distance），600s是阈值（时长or距离）
    return g

def Get_Rj(x):
    x=x.reset_index()
    Sj=x['area'][0]
    
    dt=0
    for i in range(len(x)):
        vl=x['sum_pop'][i]*Ga_1800s(x['cost'][i])
        dt=dt+vl
    return Sj/dt

def Get_Ai(x):
    x=x.reset_index()
    
    dt=0
    for i in range(len(x)):
        vl=x['Rj'][i]*Ga_1800s(x['cost'][i])
        dt=dt+vl
    return dt

In [117]:
# 10 step1 计算每个公园的Rj
# 公园面积 ➗ 该公园搜索阈以内的人口数

park_s3 = f_ODc_Gaode.groupby(by='Natureparkid').apply(Get_Rj).reset_index()
park_s3 = park_s3.rename(columns={0: 'Rj'})

f_ODc_Gaode = pd.merge(f_ODc_Gaode,park_s3 [['Natureparkid', 'Rj']], on='Natureparkid', how='left')

In [118]:
# 11 step2 计算每个pop cell的Ai
# 每个供给点popcell,搜索阈以内的Rj,即park_v1
pop_s3 = f_ODc_Gaode.groupby(by='Popid').apply(Get_Ai).reset_index()
pop_s3 = pop_s3.rename(columns={0: 'Ai'})

f_ODc_Gaode = pd.merge(f_ODc_Gaode,pop_s3[['Popid', 'Ai']], on='Popid', how='left')

In [119]:
# 12 简化
access_socre_c = f_ODc_Gaode[['Popid', 'sum_pop', 'geometry_x', 'Ai']]
access_socre_c = access_socre_c.rename(columns={'geometry_x':'geometry'})
access_socre_c.drop_duplicates(inplace=True)
len(access_socre_c)

288

In [120]:
result_c = pop_resid.merge(access_socre_c[['Popid', 'Ai']], on='Popid', how='left')

In [121]:
result_c = result_c[['sum_pop', 'geometry', 'Popid', 'Ai']]

In [122]:
result_c.to_file('tmp/result_c.geojson')